<a href="https://colab.research.google.com/github/ConnerEvans/ConnerEvans.github.io/blob/main/Process_Cluster_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

**This is where you can process all your data to reveal how *you* think these items should be clustered!**



If you did not come from the clustering portion of my website with some data copied to your clipboard, then go here: https://github.com/ConnerEvans/cluster.html to check it out and come back once you have some data.

I originally intended to calculate this all on the website, but these algorithms take a bit of time to compute. Don't worry, it is much faster using Google's hardware! 

I'm going to use this notebook to not only calculate the clusters, but to explain the entire process for those who are interested.

First, paste the data to the right of the equal sign in the box below.

Second, click on the 'Runtime' tab above and then click 'Run all'.

Lastly, if you don't want a peak under the hood to understand how I am calculating these clusters, scroll all the way to the bottom. If you do want to learn how the sausage is made, I'll explain the code and the theory behind the algorithms in the text boxes alongside the code.

In [ ]:
data = 
#      ^^^ Paste data here

print('Data Processed!')

---

>## Preparing the Data

First things first, lets load all the relevant libraries so we can take advatange of their well-made functions and data structures.

In [ ]:
import numpy as np
from sklearn import manifold
from itertools import permutations
from scipy.stats import rankdata
from numba import njit

Next, lets take the data you entered and transform it into a matrix that will be easier to use.

Each number in the matrix has a row and column that correspond to the items being compared. For example, the number in the first row and second column shows what you said about the similarity between the first and second item:

0 means you were not asked about this pair

1 means you said they are Similar

2 means you said they are Not Similar

3 means you said they are Completely Different.

In [ ]:
NOT_COMPARED = 0
SIMILAR = 1
NOT_SIMILAR = 2
COMPLETELY_DIFFERENT = 3

# Separate the first value which says what kind of items are being clustered
file_index = data[0]
similarity_data = np.array(data[1:])

# If n_items is the total number of items being clustered, 
# then the number of possible comparisons follow this equation: n_items*(n_items-1)/2
# I use the quadratic equation to solve for n_items given n_comparisons
n_comparisons = len(similarity_data)
n_items = int(np.round((1+(8*n_comparisons+1)**.5)/2))

# Put the values in the right spot
comparison_matrix = np.zeros((n_items,n_items))
index = 0
for i in 1+np.arange(n_items-1):
  for j in np.arange(i):
    comparison_matrix[i,j] = similarity_data[index]
    index += 1

# A comparison between the first and second items is the same as between the second and first
comparison_matrix += comparison_matrix.T
print("Matrix of input data:")
print(comparison_matrix)

Now that we have our data set up, I will take a not-so-breif aside to explain the theory behind the algorithms I created.



---



---



# The Theory

>## Explaining Bayesian Analysis

One of my foundational goals was to take a Bayesian approach to this, so lets start with a table that shows how Bayesian analysis works.

![picture](https://drive.google.com/uc?id=1ZI3kkYx5AY8Z1ZfJHm9lqCHu7qMJ8ObE)

This table has 10 objects. If you were to pick one at random, the probability that it would be blue [written as $P(Blue)$ or $P(B)$] would be 6 out of the 10 ojects or 0.6. The probability that it would be a square [written as $P(Square)$ or $P(A)$] would be 5 out the 10 objects or 0.5.

If I told you that the object you chose was blue, then the probability that the object was square would be 4 out of 6 objects or 0.67. This idea - the probability that the oject is a square *given* that it is blue - is written as $P(Square|Blue)$. 




This is where Bayes' theorem comes into play:

$P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B)}$

Sometimes we know $P(B|A)$, but not $P(A|B)$. A common instance of this is when we know the probability that someone will test positive for an illness if they have it, but we don't actually know the probability that they are ill if they test positive until we use Bayes' theorem to calculate it.

So back to the objects. Lets say we know that if you have an object that is Square there is a 4 in 5 or 0.8 chance it is Blue [$P(B|A)=0.8$]. We also know that the probability the object is Blue is 6 in 10 or 0.6 [$P(B)=0.6$] and the probability it is square is 5 in 10 or 0.5 [$P(A)=0.5$].

Lets use the equation to calculate the probability that the object is Blue given that we know it is a Square: $\frac{0.8 * 0.5}{0.6} = \frac{0.4}{0.6} = 0.67$. 

To better see how everything cancels out, lets leave everything as fractions: $\frac{\frac{4}{5} * \frac{5}{10}}{\frac{6}{10}}$

Now, we can better see that the 4 Blue Squares out of 5 Squares times 5 Squares out of 10 objects become 4 Blue Squares out of 10 objects. These 4 Blue Squares are then compared to the bottom which says that there are 6 Squares total out of the 10 objects. So 4 of the 6 Squares are Blue which gives us the fraction we were looking for; what is the probability that it is Blue knowing that it is a Square.

I know this process can seem needlessly complicated since we can simply look at the table and count the different objects, but what if we only had the probabilities and not the counts? Lets calculate it one more time using the table below which just has the probabilities. 

Note that the tick marks ' denote the opposite, so A' [read as *not A* or *A prime*] is everything that is not A.

![picture](https://drive.google.com/uc?id=1JAbkcs21HnYnE1Rzsa42TIk3jFeLYRe_)

In this situation, we have *Prior* knowledge that there is an equal chance the object is a Square or Circle [$P(A)=P(B)=0.5$]. These probabilities are called our Priors. 

So we know our Prior, which means that if a random object is picked, we know the likelihood that it is each shape. But what if we also know the probability that the object is Blue or Red given the Shape? [Shown in the Blue and Red rows] And what if we are then told that the object is Blue? We now have more information and can have a more informed likelihood that it is each shape. These new probabilities are called Posteriors.

I won't plug these values into the equation again, but you might have notice that there is no P(B) in the table, which we need to solve the formula. But we can actually calculate it from the information that we have. 

We know the probability that it is Blue given that it is a Square and we know the probability that it is a Square. We can therefore multiply these probabilities together to get the probability that the object is Blue AND a Square [$P(B|A)*P(A)=P(B\&A)$]. We can do the same thing to calculate the probablity that the object is Blue AND NOT a Square [$P(B\&A')$] and then add them together to get the overall probability that it is Blue, like this:

$P(B) = P(B\&A) + P(B\&A')$$ = P(B|A)*P(A) + P(B|A')*P(A')$

This means that, for a table like this, we can update the formula to be:

$P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B|A)*P(A) + P(B|A')*P(A')}$

---

>## Applying Bayesian Analysis

Now we know how to use Bayes Theorem! This might not seem very powerful, but it is the basis of an entire branch of statistics! So, how can we use this power to start clustering items? 

>>### The Set Up

First, lets look at an example of a clustering. 

![picture](https://drive.google.com/uc?id=1kYiw9hAEDHvM6Z6IpQ7MQ66uJKEz9kW0)

Let's go though most of the variables up front using this example. 

The number of clusters in a clustering is *n*, which in this case is 4. 

There are also 18 data points where the user said that a pair of items was either Similar, Not Similar, or Completely Different. 

Of the 7 pairs that were in the same cluster (the blue lines), the user said that 6 of them are similar (solid dark blue lines). Using this data, the likelihood that the user said a pair of items are Similar given that pair is in the same cluster is 6/7, which we will call *p*. 

Of the 11 pairs that were not in the same cluster (the lines that aren't blue), the user said that 6 of them were Completely Different (dotted red lines) and 4 of them were Not Similar (dashed orange lines). Using this data, the likelihood that the user said a pair of items are Completely Different given that pair is not in the same cluster is 6/11, which we will call *r*. Additionally, the likelihood that the user said a pair of items are Not Similar given that pair is not in the same cluster is 4/11, which we will call *q*.

But why do we need these probabailities? If you look at the cluster on the right with B, E, and G, you will see a common occurance where the user said that BE and EG are Similar, but BG is Not Similar. There is no way to distribute these items among the clusters such that all of the user's inputs are perfectly followed. Therefore we can't just follow all of the user's  inputs (which is to assume that *p* and *q+r* are equal to one). This is the whole reason that we need this complicated algorithm: we need to figure out which inputs are best to discard. The core of this algorithm will be creating and updating probabilities that each pair of items is in the same cluster.

Thankfully, there is one thing that we can take for granted. If a pair of items is in the same cluster, there is *0* probability that you said they are Completely Different. Basically, you are never wrong about a pair of items being Completely Different. This is because we can increase the amount of clusters until it is possible to spread out all of these pairs.

>>### Basic Probabilities Given Your Inputs

Now lets determine what the probability is that a pair of two random items are in the same cluster. This will be our our Prior.

Lets say that there are *n* clusters. Lets put one of the items in a random cluster. Then, when we go to add the second item, there is now only 1 out of n clusters where the first item is, which means that the probability that they are in the same cluster is $\frac{1}{n}$ and the probability that they are not in the same cluster is $\frac{n-1}{n}$.

We can plug all this information into a table. The 'Prior' row will have the priors that we just determined. The other three rows will show the probability that you gave an input given whether the pair is 'actually' in the same cluster [P(Similar|Same), P(Similar|Not Same), and so on].

![picture](https://drive.google.com/uc?id=1Q_1BMteA6nFllBcUlKqW-DskaNSaIqB8)

Now we can calculate the probability that a pair of items is in the same cluster given the information that you supplied using Bayes' Theorem!

$P(Same \mid Similar) = \frac{p\frac{1}{n}}{p\frac{1}{n} \, + \, (1-q-r)\frac{n-1}{n}} = \frac{p}{p \, + \, (1-q-r)(n-1)}$

$P(Same \mid Not \, Similar) = \frac{(1-p)\frac{1}{n}}{(1-p)\frac{1}{n} \, + \, q\frac{n-1}{n}} = \frac{(1-p)}{(1-p) \, + \, q(n-1)}$

$P(Same \mid Completely \, Different) = \frac{0\frac{1}{n}}{0\frac{1}{n} \, + \, r\frac{n-1}{n}} = 0$

>>### Probabilities Given a Third Item

This is a great starting point, but the probabilities for all the pairs that you said are Similar are equal, which isn't very helpful in determining which of those pairs are actually in the same cluster. But we have more information!

If you said that items A, B, and C are all similar, that is a very different situation than if you had said that A and B are similar, A and C are similar, but B and C are not Similar. The probability that A and B are in the same cluster should be higher for the first set than for the second set.

We can approoach this using Bayes Theorem again but for three items instead of two. This time lets use generic probabilities for the likelihood that each pair in a set of three (laid out in a triangle) is in the same cluster:

Let's say that *z* is the prior probability that the bottom pair of the triangle is in the same cluster. We are looking to calculate the posterior for *z*.

Let's say that *x* is the probability that the pair on the left of the triangle is in the same cluster and *y* is the probability for the pair on the right.

There are 5 possible distributions of 3 items in clusters:

They are all in the same cluster - in this case, the bottom pair is in the same cluster so our prior is *z* times the likelihood that the third item would randomly be in the same 1 out of *n* clusters that the pair is in. 

The bottom pair is in the same cluster, but the item at the top of the triangle is not - this would be *z* times the likelihood that the top item is in the other *n-1* clusters.

The left pair is in the same cluster, but the item on the right is not - in this case, since the bottom pair is not in the same cluster, our prior is *1-z* times the likelihood that the top item would randomly be in the same 1 out of *n* clusters that the item on the left is in.

The right pair is in the same cluster, but the item on the left is not - this is the exact same thing but mirrored, so it has the same prior.

None of the items are in the same cluster - the bottom pair is not in the same cluster, so our prior is *1-z* times the likelihood that the top item would randomly not be in either of the 2 out of *n* clusters that the items on the bottom are in.

Now we can make the table below and calculate the likelihood of each clustering given the probabilities about the third item. We will calculate and add together the likelihood of the first two clusterings to get our posterior that the bottom pair is in the same cluster. 

Note that these probabilities will have the same denominator, so instead of adding the whole fraction, I just added the numerators.

![picture](https://drive.google.com/uc?id=1VjAoLQn-2bNtPBmIUMQDVHjb5CpNncow)

$\frac{(z)\frac{1}{n}(x)(y) \, + \, 
(z)\frac{n-1}{n}(1-x)(1-y)}
{(z)\frac{1}{n}(x)(y) \, + \, 
(z)\frac{n-1}{n}(1-x)(1-y) \, + \, 
(1-z)\frac{1}{n}(x)(1-y) \, + \, 
(1-z)\frac{1}{n}(1-x)(y) \, + \, 
(1-z)\frac{n-2}{n}(1-x)(1-y)} =
\frac{(z)\big[(x)(y) \, + \, (1-x)(1-y)(n-1)\big]}
{(z)\big[(x)(y) \, + \, (1-x)(1-y)(n-1)\big] \, + \, 
(1-z)\big[(x)(1-y) \, + \, (1-x)(y) \, + \, (1-x)(1-y)(n-2)\big]}$ 

>>### Combining Probabilities

Now we are getting somewhere! We can update the probability that any pair is in the same cluster given the probabilities that a third item are in their same cluster!

But which third item should we choose? It would be better if we could use more than one additional item using sets of 4 or more. But unfortunately where there are 5 possible distributions of 3 items in clusters, there are 15 distributions for 4 items, 52 for 5, 203 for 6 and so on (these are the Bell numbers - https://en.wikipedia.org/wiki/Bell_number). So it is infeasible to do this with all the items in the same set. 

So we should probably just use sets of three, but then we have to decide which third object to choose. But what if we don't choose and calculate posteriors for very single other item. Then we would have many useful posteriors for the same pair. We just need a way to combine them...

So lets use Bayes Theorem again!

Lets have our prior that this pair of items are in the same cluster be *x* and lets have *y* be our new probability that we want to combine with our prior.

![picture](https://drive.google.com/uc?id=1ctyFB948tXIQtx7R5aKIYVsHJCgAOM21)

$\frac{(x)(y)}
{(x)(y) \, + \, (1-x)(1-y)}$ 

Very simple!

But what if we want to combine more than two? We can use the posterior that we calcualted as our new prior! 

Lets set *x* = $p_{1}$, and *y* = $p_{2}$:

$\frac{p_{1} p_{2}} {p_{1}p_{2} \, + \, (1-p_{1})(1-p_{2})}$ 

Now, using the same formula, lets combine $p_{3}$ with our new probability:

$\frac{\frac{p_{1}p_{2}} {p_{1}p_{2} \, + \, (1-p_{1})(1-p_{2})} p_{3}} 
{\frac{p_{1}p_{2}} {p_{1}p_{2} \, + \, (1-p_{1})(1-p_{2})}p_{3} \, + \, 
\big(1 \, - \, \frac{p_{1}p_{2}} {p_{1}p_{2} \, + \, (1-p_{1})(1-p_{2})} \big)(1-p_{3})} = 
\frac{p_{1}p_{2} p_{3}} {p_{1}p_{2}p_{3} \, + \, 
\big(p_{1}p_{2} \, + \, (1-p_{1})(1-p_{2}) \, - \, p_{1}p_{2} \big)(1-p_{3})} = \frac{p_{1}p_{2} p_{3}} 
{p_{1}p_{2}p_{3} \, + \, (1-p_{1})(1-p_{2}))(1-p_{3})}$

Do you see the pattern emerging? The general form can be written as:

$t = \Pi \, p_{i} \qquad f = \Pi \, (1-p_{i})$

$p = \frac{t}{t+f}$

where $p_{i}$ is the $i^{th}$ probability that we are combining and $\Pi$ is like $\Sigma$ but instead of adding all of the terms together, we multiply them.

This formulation will make it easier to compute which will be helpful.

So, now we can update the probability that each pair is in the same cluster. Every probability is informed by every set of three items and all the probabilities are different according to the information that you supplied!

>>### Removing Remaining Impossible Triangles

But we probably still have a problem, lets say that we use a cutoff of 0.5 to determine whether a pair should be in the same cluster or not. There are still likely many sets of three items that form an 'impossible triangle' where our probabilities say that two of the three pairs should be in the same cluster but the last pair should not be. In this case, it is still ambiguous how to assign the items to clusters.

But fear not! We already have the solution for this! We can simply update the probabilities of every pairs again using the new probabilities! And we will continue updating the probabilities until there are no 'impossible triangles' left.

>>### Other Approaches

There were multiple other Bayesian approaches that I tried. For example, intead of predicting whether pairs of items were in the same cluster by updating with sets of three, I tried to predicting the 5 clusterings for sets of three by updating with pairs. After implementing and testing this and multiple other approaches, I found that the one I explained and used here was the best at converging quickly to results that seemed right.

That's it! That's all the theory! There are just a few other optimizations and choices that I'll explain quickly.



---



---



# Implementing the Theory

>## Finding n

First, you may have noticed that I never explained how I was going to choose *n*. Although it is not elegant, I think the best way to do this is to try every reasonable value of n and pick the "best" one.

But how do we know which one is best? My goal is to minimize two things: the amount of pairs where you said they were similar but they ended up being in different clusters (a false positive, $FP$), and the amount of pairs where you said they were not similar but they ended up being in the same cluster (a false negative, $FN$). These are in contrast to true positives ($TP$), where you said a pair is Similar and they end up in the same cluster, and true negatives ($TN$), where you said a pair is Not Similar and they don't end up in the same cluster. Additionally, the number of pairs that you said are Completely Different, which should all be in different clusters, are called 'Not Error' ($NE$) because if such a pair were to be put in the same cluster it would be an error by definition.

So if we are minimizing the false positives AND false negatives, should we just minimize their sum? What if they shouldn't be equally weighted? Can we find a natural way to combine them into one value we can maximize?

To me, the most natural question I could ask in comparing clustering configurations is how likely your decisions were given each clustering. Doing this properly using Bayes' Theorem is intractable because of all of the possible clusterings (that follow Bell's Numbers), but there is an easy way to approximate it: we can multiply the likelihoods of your responses for each pair together. This is an approximation because we are treating the likelihood of each response as independent events even though they are not.

If we are going to multiply the probabilities for each pair together, lets start with all the true positives. Using the table above called "Starting Probabilities" we can see the probability is *p* for a pair in the same cluster. So, if we multiply the probabilities of all of these evets together, we get: $p^{TP}$. If we do this for each type of event and then multiply them all together, we get:

$likelihood = (p)^{TP}(1-q-r)^{FP}(q)^{TN}(1-p)^{FN}$

Since the base of these exponentials will always be between 1 and 0, the likelihood will be a very small number which will be difficult for the computer to handle. So instead of calculating the raw likelihood, lets calculate the ratio of the likelihood to the maximum likelihood.

Since your information about the amount of positives and negatives stays constant, we would get the maximum likelihood if all the False Positives and Negatives were True Positives and Negatives. This is because, if your information for each category is correct more than half of the time, then $p > 1-q-r$ and $q > 1-p$. This gives us that the maximum likelihood is:

$max \, likelihood = (p)^{TP+FP}(q)^{TN+FN}$

This makes the ratio of the likelihood to the maximum likelihood:

$likelihood \, ratio = L = \frac{(p)^{TP}(1-q-r)^{FP}(q)^{TN}(1-p)^{FN}}{(p)^{TP+FP}(q)^{TN+FN}} = \frac{(1-q-r)^{FP}(1-p)^{FN}}{(p)^{FP}(q)^{FN}}$

And what should we use for our values of *p*, *q*, and *r*? Well we can use their definition to do a deterministic calculation given the clustering:

$p = \frac{TP}{TP+FP}$

$q = \frac{TN}{TN+FN+NE}$

$r = \frac{NE}{TN+FN+NE}$

If we plug these into our previous equations we get:

$L = \frac{(1-\frac{TN}{TN+FN+NE}-\frac{NE}{TN+FN+NE})^{FP}(1-\frac{TP}{TP+FP})^{FN}}{(\frac{TP}{TP+FP})^{FP}(\frac{TN}{TN+FN+NE})^{FN}} =
\big(\frac{\frac{FN}{TN+FN+NE}}{\frac{TP}{TP+FP}}\big)^{FP} \big(\frac{\frac{FP}{TP+FP}}{\frac{TN}{TN+FN+NE}}\big)^{FN} = 
\big(\frac{FN}{TP}\big)^{FP} \big(\frac{FP}{TN}\big)^{FN} \big(\frac{TP+FP}{TN+FN+NE}\big)^{FP-FN}$

This will be the formula that we use to score each clustering so we can choose the one that most closely aligns with the data that you input.

A note: 

When we input the guess *n*, the algorithm will output a clustering. The numbers of clusters in this resultant clustering will not necessarily be the same as the number that we used for the guess. This is because the different values of *n* are only adjusting how likely items are to cluster with each other, not directly setting the number of clusters. This means that even for the best clustering, the number of clusters that clustering has might be different than the *n* that was used to find it.

>## Final Considerations

Now that we've determined how to find *n*, we need to determine what our initial guesses for *p*, *q*, and *r* should be. My first approach was to use educated guesses like 0.9, 0.6, and 0.3 respectively to calculate the clusterings, then calculated the actual values for *p*, *q*, and *r* in the resulting clusterings. Then I use these as the new guesses and recursively ran it until they were the same before and after. This worked well! 

Unfortunately this apporoach had the unforseen consequence of overriding the users input more than it needed to, because it expected the user's inputs to be partially wrong. This was very interesting, but I wanted to create clusters that agreed with the user's inputs as much as possible. So instead of using reasonable values, I set *p* and *q* to 0.99999999999 (and modified the equations assuming r was negligible but non-zero) to force the results to agree with the input data as much as possible. This worked wonderfully!

But this accentuates the issue we were already going to have where multiplying a bunch of numbers between 0 and 1 creates numbers so small that the computer will have to set them equal to zero/one or add them incorrectly. I fixed this by doing two things:

The first issue was that the probabilities I cared about (if items were in the same cluster) were approaching 1. Why is this an issue? Well as the probabilities get closer and closer to one, the computer hits its precision limit and will round the number to one. For normal numbers in python, the precision is about 16 significant figures which means that if I add 0.00000000000000006 to 0.9999999999999999 it would round to 1. This is demonstrated in in the code below.

In [ ]:
print(0.9999999999999999)
print(0.9999999999999999 + 0.000000000000000006)
print(0.9999999999999999 + 0.00000000000000006)
print(0.9999999999999999 + 0.0000000000000006)
print(0.9999999999999999 + 0.000000000000006)

So instead of calculating the probabilities that items are in the same cluster, I calculated the probabilities that they are NOT in the same cluster. This means that when it is getting very likely, the values will be approaching 0, not 1, which the computer can handle  more easily because representing a number close to zero like ${10}^{-20}$ is only 1 significant figure. 

But there is a limit to this, the smallest number that can be represented in this manner is about ${10}^{-324}$ which seems like it should be precise enough, but surprisingly it is not.

How then can we get even more precision? Thankfully this is very common problem for Bayesian calculations because, as I said, multiplying a bunch of probabilities together makes numbers very small. The standard solution is to convert all of the probabilities to log-scale.  This is simply done by taking the logarithm of the probabilities.

Lets take the number from before, ${10}^{-324}$. When we take the log of this number it becomes about -324 which is much easier for the computer to handle. Using the log-scale also has an added benefit that multiplying in the normal-scale is adding in the log-scale. The downside is that in order to add in the normal-scale the log-probabilities need to be converted to the normal scale, added, and then converted back.

Lastly, one thing that I noticed was that if there was an item that had not been marked as Similar to at least one other item, the algoritm took much longer to converge. I think that this is because it is 'trying' to place the item in clusters based off what it is not. This lead to some amazing inferences, but also some incorrect conclusions that took too long to arrive at. 

In order to account for this, lets make a mask for the items that you said are similar to at least one other item and then create a matrix with that subset of items. We will use this matrix going forward.

In [ ]:
similar2something_mask = ((comparison_matrix==SIMILAR).sum(axis=0)!=0)
updated_comparison_matrix = (comparison_matrix[similar2something_mask,:][:,similar2something_mask])

if (np.sum(similar2something_mask)<n_items):
  print('There are some items that you did not say are Similar to any other items. Input more data for better results.')

If you want the final clustering to include every item, you will need to at least continue inputting data until all the items have something that they are similar to.



---


---


# Finding the Best Clustering

These are a bunch of functions that I used to break the problem into smaller pieces. I explain the purpose of each one in the comments of the code.

The '@njit' before every function requires the function to be written in a way that is a little less stright forward, but it allows the functions to run much faster which is important since this core algorithm takes exponentially longer with more items.

In [ ]:
@njit
def count_impossible_triangles(M,threshold):
  # For each set of three items, there are three pairings.
  # Count how many sets have exactly two pairing values below the threshold.

  # Why this matters:
  # If A is in the same cluster as B and B is in the same cluster as C, 
  # then it should be impossible for A to not be in the same cluster as C 

  e = 0
  for k in np.arange(2,M.shape[0]):
    for j in np.arange(1,k):
      for i in np.arange(j):
        a = M[i,j] < threshold
        b = M[i,k] < threshold
        c = M[j,k] < threshold
        if (a&b&~c)|(a&~b&c)|(~a&b&c):
          e += 1
  return(e)


@njit
def log10_not(log_prob):
  # Convert a log probability to the standard-scale, subtract it from one and then convert it back to log-scale.
  # This will give the log probablity that the event will not happen.

  return(np.log10(1 - 10**log_prob))
  

@njit
def log10_add(log_probs):
  # Convert the log probabilities to the standard-scale in order to add them. Then convert the answer back to log-scale.
  # The largest (least negative) probability is factored out to increase the precision of the calculation.

  sorted_log_probs = np.sort(log_probs)
  if np.isinf(sorted_log_probs[-1]):
    return(np.NINF)
  return(sorted_log_probs[-1] + np.log10(1 + np.sum(10**(sorted_log_probs[:-1] - sorted_log_probs[-1]))))


def find_in_sublists(list_of_lists, value):
  # Determine which sublist the given value is in

  for sub_i, sublist in enumerate(list_of_lists):
    try:
      sublist.index(value)
      return sub_i
    except ValueError:
      pass

  raise ValueError('%s is not in lists' % value)


@njit
def P_given_n(S,n,p,q,max_propagation_cycles):
  # Calculate probability that items are NOT in the same cluster using the log probability scale.
  # This is the core algorithm 
  
  # Set up variables
  N = S.shape[0]

  prob_no_info = np.log10((n-1)/n)
  prob_similar = np.log10((n-1)*(1-q)/(p+(n-1)*(1-q)))
  prob_not_similar = np.log10((n-1)*q/((1-p)+(n-1)*q))
  prob_completely_different = np.log10(1)

  log_n1 = np.log10(n-1)
  log_n2 = np.log10(n-2)

  D = (S != 0) # Matrix: does this pair have comparison data
  P = S*0.0 # Initialize probability matrix


  # Input used data
  for i in np.arange(N):
    for j in np.arange(N):
      if (S[i,j] == NOT_COMPARED):
        P[i,j] = prob_no_info
      elif (S[i,j] == SIMILAR):
        P[i,j] = prob_similar
      elif (S[i,j] == NOT_SIMILAR):
        P[i,j] = prob_not_similar
      elif (S[i,j] == COMPLETELY_DIFFERENT):
        P[i,j] = prob_completely_different

  # If there are two items that have both been compared to a third,
  # use the third to update that pair's probability.
  # Do this for every pair with each third item.
  # For example, only update pair AB given C if the user has input data for both AC and BC 
  P_temp = P.copy()
  for j in np.arange(1,N):
    for i in np.arange(j):
      t = log10_not(P[i,j])
      f = P[i,j]
      for k in np.arange(N):
        if (k!=i)&(k!=j):
          if (D[i,k]!=0) & (D[j,k]!=0):
            not_a = log10_not(P[i,k])
            not_b = log10_not(P[j,k])

            t += log10_add(np.array([not_a + not_b, P[i,k] + P[j,k] + log_n1]))
            f += log10_add(np.array([P[i,k] + not_b, not_a + P[j,k], P[i,k] + P[j,k] + log_n2]))
      
      temp = f - log10_add(np.array([t,f]))
      P_temp[i,j] = temp
      P_temp[j,i] = temp
  P = P_temp.copy()

  # Repeatedly update the probabilities for every pair given every other third item
  # until there are 0 impossible triangles or we hit the max cycles.
  for c in np.arange(max_propagation_cycles):
    for j in np.arange(1,N):
      for i in np.arange(j):
        t = log10_not(P[i,j])
        f = P[i,j]
        for k in np.arange(N):
          if (k!=i)&(k!=j):
            not_a = log10_not(P[i,k])
            not_b = log10_not(P[j,k])

            t += log10_add(np.array([not_a + not_b, P[i,k] + P[j,k] + log_n1]))
            f += log10_add(np.array([P[i,k] + not_b, not_a + P[j,k], P[i,k] + P[j,k] + log_n2]))

        P_temp[i,j] = f - log10_add(np.array([t,f]))
        P_temp[j,i] = P_temp[i,j]

        if np.isnan(P_temp[i,j]):
          print(f,t)
    P = P_temp.copy()
    e = count_impossible_triangles(P,np.log10(.5))
    if e == 0:
      return(P)
  
  print('HIT MAX CYCLES')
  return(P)


@njit
def create_clusters(P):
  # Input a matrix of cluster probabilities with now impossible triangles
  # Output the sets of items in each cluster

  log10_half = np.log10(.5)

  clusters = np.arange(P.shape[0])
  for i in np.arange(P.shape[0]):
    for j in np.arange(i):
      if (P[i,j] < log10_half):
        clusters[clusters == j] = i

  return(clusters)


def test_clusters(P,S):
  # Calculate the number of False Positive, False Positives, etc.
  # Approximate likelihood of the input data given the clustering.

  TP = np.sum((P < np.log10(.5)) & (np.triu(S) == SIMILAR))
  FP = np.sum((P > np.log10(.5)) & (np.triu(S) == SIMILAR))
  TN = np.sum((P > np.log10(.5)) & (np.triu(S) == NOT_SIMILAR))
  FN = np.sum((P < np.log10(.5)) & (np.triu(S) == NOT_SIMILAR))
  Errors = np.sum((P < np.log10(.5)) & (np.triu(S) == COMPLETELY_DIFFERENT))

  likelihood = (FN/TP)**FP * (FP/TN)**FN * ((TP+FP)/(TN+FN))**(FP-FN)

  if (Errors > 0):
    print(Errors,'Errors')
    likelihood = 0

  return(FP,FN,likelihood)

Using the above functions, find the best clusterings and their likelihood for each value of *n* in a range and choose the one with the highest likelihood.

Note again: the variable *n* is the input guess that affects the likelihood that pairs will cluster, the variable *num_clusters* is the actual amount of clusters in the clustering that was created. 

In [ ]:
min_clusters = 3
max_clusters = 20
p = 1 - 1e-12
q = 1 - 1e-12
max_propagation_cycles = 30

likelihood_best = 0

for n in np.arange(min_clusters,max_clusters+1):
  # P a matrix of the probabilities for each pair of items
  P = P_given_n(updated_comparison_matrix,n,p,q,max_propagation_cycles)
  FP,FN,likelihood = test_clusters(P,updated_comparison_matrix)

  clusters = create_clusters(P)
  num_clusters = np.unique(clusters).size
  print('n:',n,' Number of Clusters:',num_clusters,' False Positives:',FP,' False Negatives:',FN,' Likelihood:',"%.3g" % likelihood)

  # Update the best values if the new clustering is better
  if likelihood > likelihood_best:
    likelihood_best = likelihood
    clusters_best = clusters
    num_clusters_best = num_clusters
    FP_best = FP
    FN_best = FN
    n_best = n

  # All else equal, we want the clustering where n is closest to num_clusters 
  elif (FP == FP_best)&(FN == FN_best)&(np.abs(n - num_clusters) < np.abs(n_best - num_clusters_best)):
    clusters_best = clusters
    num_clusters_best = num_clusters
    n_best = n


  # These shouldn't be triggered because I set the range of n to be so wide,
  # but if the resulting number of cluster for the best clustering hasn't been tested, 
  # it should be tested.
  if num_clusters < min_clusters:
    print('Lower Min Clusters to',num_clusters)
  elif num_clusters > max_clusters:
    print('Raise Max Clusters to',num_clusters)

print()
print('Best n:',n_best,' Number of Clusters:',num_clusters_best,' False Positives:',FP_best,' False Negatives:',FN_best,' Likelihood:',"%.3g" % likelihood_best)

Feel free to change the priors *p* and *q* in the code above to see how they affect the final result!

# Displaying the Best Clustering

We did it! We found the best way to group up all the items given *your* input!

Now what? Well I could just show you which items are in each cluster, but where's the fun in that? We have so much data! Let's use it!

Note: though I think we can use the data to make the visualization more informative, I do not think there is a rigorous way to do it like there was for the previous section, so I will make more decisions without a strong reason in this section.

First, what is the general layout going to be?

Let's have the clusters be in a line from left to right and have the items inside of the clusters be in a line from top to bottom.

This gives us some flexibility in ordering the clusters and items within each cluster, but won't be too visually complicated.

>## Cleaning Clusters

In order to make the clusters legible, and to make some of the methods I used computationally feasible, we want no more than 9 clusters. This can usually be resolved easily by removing the clusters that have just one item in them. The only downside of this is that we won't see where it would be placed among the clusters, but with only one item, we probably wouldn't have enough data to place it in a proper place anyway. In the unlikely event that we still have too many clusters after removing the ones with only one item, we can remove the clusters with two, and then three, until we have less than 9.

In [ ]:
_, clusters, counts = np.unique(clusters_best, return_inverse = True, return_counts = True)
num_clusters = len(counts)

n_new_clusters = (n_items - np.sum(similar2something_mask))
full_clusters = np.zeros(n_items, dtype=int)
full_clusters[similar2something_mask] = clusters
full_clusters[~similar2something_mask] = np.arange(num_clusters, num_clusters + n_new_clusters)
num_clusters +=  n_new_clusters
counts = np.concatenate((counts,np.ones(n_new_clusters)))

removal_size = 1
cluster_0_is_removed_items = False
while num_clusters > 9:
  remove_clusters = np.arange(num_clusters)[(counts == removal_size)]
  removal_mask = np.any((full_clusters[:,np.newaxis] == remove_clusters),axis=1)
  full_clusters[removal_mask] = -1
  num_clusters -= np.sum(counts == removal_size)
  removal_size += 1
  cluster_0_is_removed_items = True

_, clusters, counts = np.unique(full_clusters, return_inverse = True, return_counts = True) 
# the 0 cluster now contains all of the clusters that were removed
num_clusters = len(counts)

print('Clustering:')
print(clusters)

Note: I first added back all of the items that weren't Similar to anything by giving them each their own clusters. If this pushes the number of clusters over 9, then they will just be removed. I do this because, if there aren't many items, a significant portion of them might be in single-item clusters and I want the final visualization to show most of the items.

If you want to remove clusters based on their size, regardless of how many clusters remain, change the variable 'removal_threshold' from 0 to 1 or 2 or whatever size clusters you want to remove and then click on the 'Runtime' tab above and click 'Run after'. This will remove clusters of that size and smaller and then recalculate the rest.

In [ ]:
removal_threshold = 0

remove_clusters = np.arange(num_clusters)[(counts <= removal_threshold)]
removal_mask = np.any((clusters[:,np.newaxis] == remove_clusters),axis=1)
clusters[removal_mask] = -1

if np.sum(counts <= removal_threshold)>0:
  cluster_0_is_removed_items = True

_, clusters = np.unique(clusters, return_inverse = True)
num_clusters = max(clusters) + 1

>## Ordering Clusters

Now that we only have the clusters we want to use, we can order them.

Let's create some sort of similarity value between each cluster that we can use to determine which clusters should be closer or further from each other. For example, if two clusters have multiple pairs of items between them that you said are Completely Different, those should be further apart than two clusters that don't have any pairs of items between them that you said are completely different.

We already have a method for calculating the probability that two items are in the same cluster. What if we just combine the probabilities for each pair between two clusters? For example if there are two clusters with items A and B and then C, D, and E, we can calculate and combine the probabilities for AC, AD, AE, BC, BD, and BE. This would allow two clusters with more 'Similars' and less 'Completely Different' to have a higher value for their similarity score!

But if we say that the probability for two items that are Completely Different is 0, then it will automatically set the overall probability to 0 which would make us lose the differentiation. Therefore, we need to make the probability for two items that are Completely Different greater than 0.

Before, when I chose the values for *p* and *q*, I set them arbitrarily close to 1 for the sake of the algorithm, but now we can just use the deterministic calculation for them using the actual clustering. And, with this method, in order to make items that are Completely Different not have a probability of 0, we will just add 1 data point that is in the same cluster but marked as Completely Different.

First, let's calculate these variables and the posteriors that two items are in the same cluster given whether they were labeled Similar, Not Similar, or Completely Different.

In [ ]:
sim = np.sum(np.triu(updated_comparison_matrix) == SIMILAR)
nsim = np.sum(np.triu(updated_comparison_matrix) == NOT_SIMILAR)
CD = np.sum(np.triu(updated_comparison_matrix) == COMPLETELY_DIFFERENT)

TP = sim - FP_best + FN_best + 1
TN = nsim - FN_best + FP_best + CD

p = (TP-FP_best)/TP
q = (TN-FN_best-CD)/TN
r = CD/TN
e = 1/TN

n = num_clusters
if cluster_0_is_removed_items:
  n -= 1

similar_posterior = np.log10(p/(p+(1-q-r)*(n-1)))
not_similar_posterior = np.log10((1-p)/((1-p)+q*(n-1)))
completely_different_posterior = np.log10(e/(e+r*(n-1)))

Now, let's combine these probabilities using 1/n for our prior that two clusters are the same. 

I found that using the log-probabiilities instead of the normal probabilities for the similarity scores resulted in better orderings. I think that it was better because it accentuated the penalty for clusters being dissimilar, which seems especially important because these different clusters should have low similarity scores with each other.

The similarity scores are stored in the Matrix Q

In [ ]:
Q = np.log10(np.ones((n,n))/n)
prior = np.log10(1/n)
log_n1 = np.log10(n-1)

for cluster_i in np.arange(1,num_clusters):
  for cluster_j in np.arange(cluster_i):
    interaction = comparison_matrix[clusters==cluster_i,:][:,clusters==cluster_j]
    
    n_p = np.sum(interaction==SIMILAR)
    n_q = np.sum(interaction==NOT_SIMILAR)
    n_r = np.sum(interaction==COMPLETELY_DIFFERENT)

    t = prior + similar_posterior*n_p + not_similar_posterior*n_q + completely_different_posterior*n_r #logscale
    f = log10_not(prior) + log10_not(similar_posterior)*n_p + log10_not(not_similar_posterior)*n_q + log10_not(completely_different_posterior)*n_r + log_n1
    
    Q[cluster_i,cluster_j] = t - log10_add(np.array([t,f]))

Q += Q.T
if cluster_0_is_removed_items:
  Q = Q[1:,1:]

print('Similarity Scores:')
print(np.round(Q,3))

Now we have similarity scores for each set of two clusters, so lets create an overall score for different orderings so we can pick the best ordering. To calculate the score for orderings, lets do a weighted sum of the similarity scores where the weights are higher if two clusters are closer to each other in the ordering. This will make the clusters that are the most similar next to each other. I print out the weight matrix W to show you how I weighted it.

I was originally going to use an algorithm that varied the ordering a little at a time to find the best one, so that I didn't have to check literally every possible ordering. In practice, the orderings I got were strongly affected by the initial guess and usually changed each time I ran it. I tried to take the best of many runs, but there was still a decent amount of variability, so I decided to check every possible ordering to find the best one so that there is no randomness. But checking every ordering grows exponentially with the number of clusters, which is why the number of clusters cannot be over 9.

In [ ]:
# num_clusters cannot be over 9 or this will take too long
perms = permutations(range(Q.shape[0]), int(Q.shape[0]))
W = np.triu(num_clusters - np.triu(np.ones((num_clusters,num_clusters)))@np.triu(np.ones((num_clusters,num_clusters))))
np.fill_diagonal(W,0)
print('Weight Matrix:')
print(W)

best_score = np.NINF
for perm in perms:
  score = np.sum(Q[:,perm][perm,:]*W)
  if score > best_score:
    best_score = score
    best_perm = perm

if cluster_0_is_removed_items:
  best_perm += 1
  
print()
print('Best Ordering:')
print(best_perm)

Now we have a good ordering of our Clusters from left to right! Theoretically there should be something descriptive about the items in the clusters that varies from one extreme to the other as you go from left to right.

>## Visualizing Each Cluster

Now lets look at the items inside each cluster. Similar to how we created a meaningful ordering of the clusters from left to right, we can order the items within the clusters from top to bottom. Theoretically there should be something descriptive about the items that varies from one extreme to the other as you go from top to bottom.

We *could* create this ordering within each cluster the same way we ordered the clusters: by weighting the similarity scores for every ordering. Instead, I want to use Multi-dimensional Scaling (https://scikit-learn.org/stable/modules/manifold.html#multidimensional-scaling) so that we can input distances based on the similarity scores and output coordinates for each item. This algorithm will try to find the coordinates in a given number of dimensions that minimize the difference between the actual distances and the ones given to it. So if the number of dimensions is one, it will find the best numbers on a number line to minimize that difference; if the number of dimensions is two, then it would be finding the (x,y) values on a graph that minimize that difference.

By using this method instead of the previous one, we won't have to limit the number of items in each cluster *and* we will be able to find which item in each cluster most epitomizes that cluster. To order the clusters we will have the algorithm create coordinates in one dimension and the ordering of the coordinates will be the ordering of the items. To find the item that exemplifies the cluster we will create coordinates in a number of dimensions equal to the number of items in that cluster. This way, the distances will be able to be as accurate as possible. Then, in this high dimensional space, we can see which item has coordinates closest to the center and say that this one embodies the cluster.

I found, through experimentation, that the best way to get distances from the similarity scores is just to square the log-probabilities. This gives good distance values that consistently push pairs of items that are Not Similar further apart than ones that have no data.

We will use the same values for *prior*, *similar_posterior*, *not_similar_posterior*, and *completely_different_posterior* that we calculated above.

In [ ]:
visualization = list()
vis_info = list()

if cluster_0_is_removed_items:
  visualization.append(np.arange(n_items)[clusters==i])
  vis_info.append(1)
else:
  visualization.append([])
  vis_info.append(0)


j = 0
for i in best_perm:
  j += 1
  print('Cluster ',j,' of ',num_clusters)

  items_in_cluster_i = np.arange(n_items)[clusters==i]
  items_data = comparison_matrix[clusters==i,:][:,clusters==i]


  if len(items_in_cluster_i) > 2:
    P = items_data*0   # probability matrix
    P[items_data == NOT_COMPARED] = prior
    P[items_data == SIMILAR] = similar_posterior
    P[items_data == NOT_SIMILAR] = not_similar_posterior
    P[items_data == COMPLETELY_DIFFERENT] = completely_different_posterior
    np.fill_diagonal(P,0)

    mds = manifold.MDS(
        n_components=len(items_in_cluster_i),
        metric=True,
        max_iter=3000,
        eps=1e-12,
        dissimilarity="precomputed",
        n_init=1,
    )
    full_dimension_coordinates = mds.fit_transform(P**2)

    vis_info.append(items_in_cluster_i[np.sum((full_dimension_coordinates-np.mean(full_dimension_coordinates,axis=0))**2,axis=1).argmin()])


    mds = manifold.MDS(
        n_components=1,
        metric=True,
        max_iter=3000,
        eps=1e-12,
        dissimilarity="precomputed",
        n_init=1,
    )
    one_dimension_coordinates = mds.fit_transform(P**2)[:,0]
    cluster_ordering = np.argsort(one_dimension_coordinates)

    visualization.append(list(items_in_cluster_i[cluster_ordering]))

  else:
    vis_info.append(-1)
    visualization.append(list(items_in_cluster_i))




# Conclusion

We're done! If you have been reading along, I hope you enjoyed seeing my thought process for making these clusters out of such simple inputs. But now I've made you wait long enough! Let there be no further delay in you actually getting to see your clusters! 

Let's just output the data so you can copy and paste it back on the website to finally see the results. Make sure to use the link I give you because you can't access this page without it!

In [ ]:
print(file_index,';')
print(vis_info,';')
print(visualization)

Copy everything directly above and paste it here: https://ConnerEvans.github.io/cluster_visualization.html

Note:

In [ ]:
if (np.sum(similar2something_mask)<n_items):
  print('There are some items that you did not say are Similar to any other items. Input more data for better results.')
else:
  print('The clusters will always get more accurate with more data, but you have input enough results for every item to be considered in the clustering.')

Enjoy your clusters!